In [6]:
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web
import pickle
import requests

Grab Company info from Wiki and save them locally

In [7]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)

    return tickers

In [ ]:
_ = save_sp500_tickers()

In [53]:
existed_ticker = []

### Grap stock data and save them locally as CSV file.

In [54]:
def get_data_from_google(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle","rb") as f:
            tickers = pickle.load(f)

    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
    
    start = dt.datetime(2000, 1, 1)
    end = dt.datetime(2016, 12, 31)
    
    for idx, ticker in enumerate(tickers):
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            try:
                df = web.DataReader(ticker, "google", start, end)
                df.to_csv('stock_dfs/{}.csv'.format(ticker))
                existed_ticker.append(ticker)
            except:
                print(ticker + " not existed in Google finance")
                continue
        else:
            print('Already have {}'.format(ticker))
            existed_ticker.append(ticker)
        if idx%10==0:
            print("finised {}/{}".format(idx, len(tickers)))

In [55]:
get_data_from_google()

Already have MMM
finised 0/505
Already have ABT
Already have ABBV
Already have ACN
Already have ATVI
Already have AYI
Already have ADBE
Already have AMD
Already have AAP
Already have AES
Already have AET
finised 10/505
Already have AMG
Already have AFL
Already have A
Already have APD
Already have AKAM
Already have ALK
Already have ALB
Already have ARE
Already have ALXN
Already have ALGN
finised 20/505
Already have ALLE
Already have AGN
Already have ADS
Already have LNT
Already have ALL
Already have GOOGL
Already have GOOG
Already have MO
Already have AMZN
Already have AEE
finised 30/505
Already have AAL
Already have AEP
Already have AXP
Already have AIG
Already have AMT
Already have AWK
Already have AMP
Already have ABC
Already have AME
Already have AMGN
finised 40/505
Already have APH
Already have APC
Already have ADI
Already have ANSS
Already have ANTM
Already have AON
Already have APA
Already have AIV
Already have AAPL
Already have AMAT
finised 50/505
Already have ADM
Already have A

### combine all adjust price of different stock into one csv file.

In [68]:
def compile_data():
    with open("sp500tickers.pickle","rb") as f:
        tickers = pickle.load(f)
        
    main_df = pd.DataFrame()
    
    for count, ticker in enumerate(existed_ticker):
        print(ticker)
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)
        df.rename(columns={'Close':ticker}, inplace=True)
        
        df.drop(['Open','High','Low','Volume'],1,inplace=True)
        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')

        if count % 10 == 0:
            print(count)
    print(main_df.head())
    main_df.to_csv('sp500_joined_closes.csv')

In [69]:
compile_data()

MMM
0
ABT
ABBV
ACN
ATVI
AYI
ADBE
AMD
AAP
AES
AET
10
AMG
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
20
ALLE
AGN
ADS
LNT
ALL
GOOGL
GOOG
MO
AMZN
AEE
30
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
40
APH
APC
ADI
ANSS
ANTM
AON
APA
AIV
AAPL
AMAT
50
ADM
ARNC
AJG
AIZ
T
ADSK
ADP
AN
AZO
AVB
60
AVY
BHI
BLL
BAC
BK
BCR
BAX
BBT
BDX
BBBY
70
BRK.B
BBY
BIIB
BLK
HRB
BA
BWA
BXP
BSX
BMY
80
AVGO
BF.B
CHRW
CA
COG
CPB
COF
CAH
CBOE
KMX
90
CCL
CAT
CBG
CBS
CELG
CNC
CNP
CTL
CERN
CF
100
SCHW
CHTR
CHK
CVX
CMG
CB
CHD
CI
XEC
CINF
110
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
COH
KO
120
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
COO
130
GLW
COST
COTY
CCI
CSRA
CSX
CMI
CVS
DHI
DHR
140
DRI
DVA
DE
DLPH
DAL
XRAY
DVN
DLR
DFS
DISCA
150
DISCK
DISH
DG
DLTR
D
DOV
DOW
DPS
DTE
DD
160
DUK
DXC
ETFC
EMN
ETN
EBAY
ECL
EIX
EW
EA
170
EMR
ETR
EVHC
EOG
EQT
EFX
EQIX
EQR
ESS
EL
180
ES
RE
EXC
EXPE
EXPD
ESRX
EXR
XOM
FFIV
FB
190
FAST
FRT
FDX
FIS
FITB
FE
FISV
FLIR
FLS
FLR
200
FMC
FL
F
FTV
FBHS
BEN
FCX
GPS
GRMN
IT
210
GD
GE
GGP
GIS
GM
GPC
GILD
GPN
GS
GT
220
GWW
HAL
HB